In [1]:
import warnings
import torch
from transformers import AutoTokenizer
from transformers import DistilBertForTokenClassification
import torch.nn as nn
import re
warnings.filterwarnings('ignore')

In [2]:
def align_word_ids(texts):
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

In [3]:
def evaluate_one_text(model, sentence):
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    unique_labels2 = set()
    unique_labels2.add('I-TR')
    unique_labels2.add('I-PR')
    unique_labels2.add('B-TR')
    unique_labels2.add('I-TE')
    unique_labels2.add('B-PR')
    unique_labels2.add('O')
    unique_labels2.add('B-TE')
    print(unique_labels2)
    ids_to_labels2 = {v: k for v, k in enumerate(sorted(unique_labels2))}
    print(ids_to_labels2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length=512,
                     truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels2[i] for i in predictions]
    return prediction_label

In [4]:
class DistilbertNER(nn.Module):
    def __init__(self, tokens_dim):
        super(DistilbertNER, self).__init__()
        if type(tokens_dim) != int:
            raise TypeError('Please tokens_dim should be an integer')
        if tokens_dim <= 0:
            raise ValueError(
                'Classification layer dimension should be at least 1')
        self.pretrained = DistilBertForTokenClassification.from_pretrained(
            "distilbert-base-uncased",
            num_labels=tokens_dim)  # set the output of each token classifier = unique_lables

    def forward(self, input_ids, attention_mask,
                labels=None):  # labels are needed in order to compute the loss

        if labels is None:
            out = self.pretrained(input_ids=input_ids,
                                  attention_mask=attention_mask)
            return out

        out = self.pretrained(input_ids=input_ids,
                              attention_mask=attention_mask, labels=labels)
        return out

In [5]:
class NLPmodule:
    def __init__(self):
        self.model = torch.load("./NER_model", "cpu")

    def append_element(self, idx, tag1, pr2, te2, tr2):
        if tag1 == 'B-PR' or tag1 == 'I-PR':
            pr2.append(idx)
        elif tag1 == 'B-TE' or tag1 == 'I-TE':
            te2.append(idx)
        elif tag1 == 'B-TR' or tag1 == 'I-TR':
            tr2.append(idx)

    def get_ner_token(self, text):
        text = ' '.join(text)
        label = evaluate_one_text(self.model, text)

        for i in range(len(label)):
            if (i == 0) and (label[i] == 'I-PR'):
                label[i] = 'B-PR'
            elif (i == 0) and (label[i] == 'I-TE'):
                label[i] = 'B-TE'
            elif (i == 0) and (label[i] == 'I-TR'):
                label[i] = 'B-TR'
            elif (i != 0) and (label[i] == 'I-PR') and (
                    label[i - 1] != 'B-PR' and label[i - 1] != 'I-PR'):
                label[i] = 'B-PR'
            elif (i != 0) and (label[i] == 'I-TE') and (
                    label[i - 1] != 'B-TE' and label[i - 1] != 'I-TE'):
                label[i] = 'B-TE'
            elif (i != 0) and (label[i] == 'I-TR') and (
                    label[i - 1] != 'B-TR' and label[i - 1] != 'I-TR'):
                label[i] = 'B-TR'
        # post-processing

        print(text)
        print(label)
        pattern = "[ ]"
        words = re.split(pattern, text)
        for i in range(len(words)):
            words[i] += "="
        maps = list(map(str.__add__, words, label))
        print(maps)

        # change output format
        tagging = []
        pr = []
        te = []
        tr = []
        pr2 = []
        te2 = []
        tr2 = []

        B_tag = ['B-PR','B-TE','B-TR']
        I_tag = ['I-PR','I-TE','I-TR']
        for idx in range(len(maps)):
            if idx < len(maps) - 1:
                word1, tag1 = maps[idx].split("=")
                word2, tag2 = maps[idx + 1].split("=")
                if tag1 in B_tag:
                  self.append_element(idx, tag1, pr2, te2, tr2)
                  if tag2 not in I_tag:
                    self.append_element(idx, tag1, pr2, te2, tr2)
                elif (tag1 in I_tag) and (tag2 not in I_tag):
                  self.append_element(idx, tag1, pr2, te2, tr2)
            else:
                word1, tag1 = maps[idx].split("=")
                if tag1 in B_tag:
                  self.append_element(idx, tag1, pr2, te2, tr2)
                  self.append_element(idx, tag1, pr2, te2, tr2)
                elif tag1 in I_tag:
                  self.append_element(idx, tag1, pr2, te2, tr2)
            tagging.append(tag1)

        for i in range(0, len(pr2), 2):
            pr.append(tuple(pr2[i:i + 2]))
        for i in range(0, len(te2), 2):
            te.append(tuple(te2[i:i + 2]))
        for i in range(0, len(tr2), 2):
            tr.append(tuple(tr2[i:i + 2]))
        
        print(tagging)
        print("PR =", pr)
        print("TE =", te)
        print("TR =", tr)

In [6]:
instance = NLPmodule()
input_list = ['Since', 'this', 'patient', 'suffered', 'a', 'heart', 'attack', 'and', 'has', 'been', 'wearing', 'a', 'pacemaker', ',', 'an', 'electromagnetic', 'resonance', 'scan', 'cannot', 'be', 'performed', 'to', 'diagnose', 'lung', 'cancer', '.']
print(input_list)

instance.get_ner_token(input_list)

['Since', 'this', 'patient', 'suffered', 'a', 'heart', 'attack', 'and', 'has', 'been', 'wearing', 'a', 'pacemaker', ',', 'an', 'electromagnetic', 'resonance', 'scan', 'cannot', 'be', 'performed', 'to', 'diagnose', 'lung', 'cancer', '.']
{'O', 'B-PR', 'B-TE', 'I-TE', 'I-PR', 'I-TR', 'B-TR'}
{0: 'B-PR', 1: 'B-TE', 2: 'B-TR', 3: 'I-PR', 4: 'I-TE', 5: 'I-TR', 6: 'O'}
Since this patient suffered a heart attack and has been wearing a pacemaker , an electromagnetic resonance scan cannot be performed to diagnose lung cancer .
['O', 'O', 'O', 'O', 'B-PR', 'I-PR', 'I-PR', 'O', 'O', 'O', 'O', 'B-TR', 'I-TR', 'O', 'B-TE', 'I-TE', 'I-TE', 'I-TE', 'O', 'O', 'O', 'O', 'O', 'B-PR', 'I-PR', 'O']
['Since=O', 'this=O', 'patient=O', 'suffered=O', 'a=B-PR', 'heart=I-PR', 'attack=I-PR', 'and=O', 'has=O', 'been=O', 'wearing=O', 'a=B-TR', 'pacemaker=I-TR', ',=O', 'an=B-TE', 'electromagnetic=I-TE', 'resonance=I-TE', 'scan=I-TE', 'cannot=O', 'be=O', 'performed=O', 'to=O', 'diagnose=O', 'lung=B-PR', 'cancer=I-PR